In [94]:
from config import api_key
import requests
import pandas as pd
from pprint import pprint
from csv_movie_data import movies_iterate

ImportError: cannot import name 'movies_iterate' from 'csv_movie_data' (C:\Users\Sam\Desktop\EPL_Project\Group-6-ETL-Project\csv_movie_data.py)

In [95]:
import os
import pandas as pd

movie_path = os.path.join("Resources", 'MoviesOnStreamingPlatforms_updated.csv')
movies_db = pd.read_csv(movie_path, index_col= False)

columns = ['film', 'year']

titles = []
years = []
for index, rows in movies_db[:20].iterrows():
    movies_list = rows.Title
    movies_year = rows.Year
    
    titles.append(movies_list)
    years.append(movies_year)

titles = []
years = []
for index, rows in movies_db[:20].iterrows():
    movies_list = rows.Title
    movies_year = rows.Year
    
    titles.append(movies_list)
    years.append(movies_year)

movie_data = list(zip(titles, years))
movies_iterate = pd.DataFrame(movie_data, columns = columns)
movies_iterate

,film,year
0,Inception,2010
1,The Matrix,1999
2,Avengers: Infinity War,2018
3,Back to the Future,1985
4,"The Good, the Bad and the Ugly",1966
5,Spider-Man: Into the Spider-Verse,2018
6,The Pianist,2002
7,Django Unchained,2012
8,Raiders of the Lost Ark,1981
9,Inglourious Basterds,2009


In [123]:
response = requests.get('https://api.themoviedb.org/3/search/movie?api_key=88871dfbaa8a1253cf888ce1feba74c4&language=en-US&query=Inception&year=2010')
response.json()['results'][0]

{'popularity': 40.338,
 'vote_count': 27411,
 'video': False,
 'poster_path': '/9gk7adHYeDvHkCSEqAvQNLV5Uge.jpg',
 'id': 27205,
 'adult': False,
 'backdrop_path': '/s3TBrRGB1iav7gFOCNx3H31MoES.jpg',
 'original_language': 'en',
 'original_title': 'Inception',
 'genre_ids': [28, 12, 878],
 'title': 'Inception',
 'vote_average': 8.3,
 'overview': 'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.',
 'release_date': '2010-07-15'}

In [140]:
def get_stuff(film, year, api_key=api_key):
    try:
        return requests.get('https://api.themoviedb.org/3/search/movie?api_key=' + api_key + '&language=en-US&query='+ film + '&year='+ str(year)).json()['results'][0]['id']
    except Exception as e:
        print(e)
        return None
id_list = []
for film, year in zip(movies_iterate['film'].tolist(), movies_iterate['year']):
    id_list.append(get_stuff(film, year))
    
# print(id_list)

movies_iterate['id'] = id_list
# for index, row in movies_iterate.iterrows():
#     response = requests.get('https://api.themoviedb.org/3/search/movie?api_key=' + api_key + '&language=en-US&query='+ row['film'] + '&year='+ str(row['year']))
#     movies_iterate['id'] = (response.json()['results'][0]['id'])

movies_iterate.head(1000)

,film,year,id
0,Inception,2010,27205
1,The Matrix,1999,603
2,Avengers: Infinity War,2018,299536
3,Back to the Future,1985,105
4,"The Good, the Bad and the Ugly",1966,429
5,Spider-Man: Into the Spider-Verse,2018,324857
6,The Pianist,2002,423
7,Django Unchained,2012,68718
8,Raiders of the Lost Ark,1981,85
9,Inglourious Basterds,2009,16869


In [141]:
response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2017&sort_by=revenue.desc')

In [142]:
highest_revenue = response.json()
highest_revenue = highest_revenue['results']
highest_revenue[0]

{'popularity': 61.804,
 'vote_count': 11145,
 'video': False,
 'poster_path': '/kOVEVeg59E0wsnXmF9nrh6OmWII.jpg',
 'id': 181808,
 'adult': False,
 'backdrop_path': '/5Iw7zQTHVRBOYpA0V6z0yypOPZh.jpg',
 'original_language': 'en',
 'original_title': 'Star Wars: The Last Jedi',
 'genre_ids': [28, 12, 878],
 'title': 'Star Wars: The Last Jedi',
 'vote_average': 6.9,
 'overview': 'Rey develops her newly discovered abilities with the guidance of Luke Skywalker, who is unsettled by the strength of her powers. Meanwhile, the Resistance prepares to do battle with the First Order.',
 'release_date': '2017-12-13'}

In [157]:
columns = ['film', 'revenue','budget']
df = pd.DataFrame(columns=columns)
df.head()

,film,revenue,budget


In [144]:
for film in movies_iterate:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df.loc[len(df)]=[film['title'],film_revenue['revenue']] # store title and revenue in our dataframe

In [150]:
type(movies_iterate['id'])

pandas.core.series.Series

In [158]:
for index, row in movies_iterate.iterrows():
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(row['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df.loc[len(df)]=row['film'],film_revenue['revenue'],film_revenue['budget'] # store title and revenue in our dataframe

In [163]:
df['pnl'] = df['revenue'] - df['budget']
df['gm'] = df['pnl']/df['budget']
df

,film,revenue,budget,pnl,gm
0,Inception,825532764,160000000,665532764,4.15958
1,The Matrix,463517383,63000000,400517383,6.35742
2,Avengers: Infinity War,2046239637,300000000,1746239637,5.8208
3,Back to the Future,381109762,19000000,362109762,19.0584
4,"The Good, the Bad and the Ugly",25118063,1200000,23918063,19.9317
5,Spider-Man: Into the Spider-Verse,375540831,90000000,285540831,3.17268
6,The Pianist,120072577,35000000,85072577,2.43065
7,Django Unchained,425368238,100000000,325368238,3.25368
8,Raiders of the Lost Ark,389925971,18000000,371925971,20.6626
9,Inglourious Basterds,321455689,70000000,251455689,3.59222
